# Import

In [ ]:
import os
import skimage.io
import csv
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from zipfile import ZipFile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from IoU import bb_intersection_over_union

# Example

In [ ]:
fname = 'airplane_500'
img = skimage.io.imread(fname=f'data/Images/{fname}.jpg')
fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(img)

with open(f'data/Airplanes_Annotations/{fname}.csv') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ')
    spamreader.__next__()
    ground_true = []
    for row in spamreader:
        x0, y0, x1, y1 = map(int, row)
        ground_true.append((x0, y0, x1, y1))
        bbox = mpatches.Rectangle((x0, y0), (x1-x0), (y1-y0), fill=False, edgecolor='red', linewidth=1)
        ax.add_patch(bbox)

plt.axis('off')
plt.show()

### Selective Search - Python

In [ ]:
from selective_search import selective_search, box_filter

boxes = selective_search(img, mode='fast')
boxes_filter = box_filter(boxes, min_size=10, topN=80)

fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(img)
for x, y, width, height in boxes_filter:
    bbox = mpatches.Rectangle((x, y), width, height, fill=False, edgecolor='red', linewidth=1)
    ax.add_patch(bbox)

plt.axis('off')
plt.show()

### Selective Search - OpenCV

In [ ]:
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
ss.setBaseImage(img)
ss.switchToSelectiveSearchFast()
ssresults = ss.process()

fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(img)
for x, y, width, height in ssresults:
    bbox = mpatches.Rectangle((x, y), width, height, fill=False, edgecolor='red', linewidth=1)
    ax.add_patch(bbox)
    
plt.axis('off')
plt.show()

# Load Data

In [ ]:
if not os.path.exists('data'):
    os.mkdir('data')
    with ZipFile('Airplanes_Annotations.zip', 'r') as zf:
        zf.extractall('data')
    with ZipFile('Images.zip', 'r') as zf:
        zf.extractall('data')

In [ ]:
train_images = []
train_labels = []
test_images = []
test_gt_values = []
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
MAX_COUNT = 10

for image, annot in zip(os.listdir('data/Images'), os.listdir('data/Airplanes_Annotations')):
    img = skimage.io.imread(fname=f'data/Images/{image}')
    with open(f'data/Airplanes_Annotations/{annot}') as csvfile:
        csvreader = csv.reader(csvfile, delimiter=' ')
        csvreader.__next__()
        ground_truth = [tuple(map(int, row)) for row in csvreader]

    if image.startswith("airplane"):
        ss.setBaseImage(img)
        ss.switchToSelectiveSearchFast()
        ssresults = ss.process()

        positive_count = 0
        negative_count = 0
        for gt in ground_truth:
            for x, y, width, height in ssresults:
                iou = bb_intersection_over_union(gt, (x, y, x+width, y+height))
                if positive_count < MAX_COUNT and iou > 0.7:
                    img_resized = cv2.resize(img[y:y+height, x:x+width], (224, 224), interpolation=cv2.INTER_AREA)
                    train_images.append(img_resized)
                    train_labels.append(1)
                    positive_count += 1
                elif negative_count < MAX_COUNT and iou < 0.3:
                    img_resized = cv2.resize(img[y:y+height, x:x+width], (224, 224), interpolation=cv2.INTER_AREA)
                    train_images.append(img_resized)
                    train_labels.append(0)
                    negative_count += 1
                elif positive_count >= MAX_COUNT and negative_count >= MAX_COUNT:
                    continue
    else:
        test_images.append(img)
        test_gt_values.append(ground_truth)

train_images = np.array(train_images)
train_labels = np.array(train_labels)

# Trainning

In [ ]:
# Model
model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=True)
for layer in (model.layers)[:15]:
    layer.trainable = False
output = tf.keras.layers.Dense(1, activation="sigmoid")(model.layers[-2].output)
model_final = tf.keras.Model(inputs=model.input, outputs=output)

# Fine-tunning
model_final.compile(
    optimizer=tf.keras.optimizers.Adam(lr=1e-4),
    loss='binary_crossentropy',
    metrics=["accuracy"]
)

checkpoint = ModelCheckpoint("Model", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')

early = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

datagen = ImageDataGenerator(
    rescale= 1./255,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=90,
    validation_split=0.2
)

history = model_final.fit_generator(
    datagen.flow(train_images, train_labels, subset='training'),
    steps_per_epoch=10,
    epochs=1000,
    validation_data=datagen.flow(train_images, train_labels,subset='validation'),
    validation_steps=3,
    callbacks=[checkpoint, early]
)

# Predict

In [ ]:
i = 8
img = test_images[i]
ground_true = test_gt_values[i]

fig, ax = plt.subplots(figsize=(6, 6))
for x0, y0, x1, y1 in ground_true:
    bbox = mpatches.Rectangle((x0, y0), (x1-x0), (y1-y0), fill=False, edgecolor='red', linewidth=1)
    ax.add_patch(bbox)

ax.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
load_model = tf.keras.models.load_model('Model')
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
ss.setBaseImage(img)
ss.switchToSelectiveSearchFast()
ssresults = ss.process()
fig, ax = plt.subplots(figsize=(6, 6))

for x, y, width, height in ssresults:
    img_resized = cv2.resize(img[y:y+height, x:x+width], (224, 224), interpolation=cv2.INTER_AREA)
    img_resized = img_resized / 255.
    img_resized = np.expand_dims(img_resized, axis=0)
    pred = load_model.predict(img_resized)
    if pred > 0.5:
        bbox = mpatches.Rectangle((x, y), width, height, fill=False, edgecolor='red', linewidth=1)
        ax.add_patch(bbox)

ax.imshow(img)
plt.axis('off')
plt.show()